In [1]:
from utils import *
from dataload import TOPLEVEL, EpochsDataset, EuclideanAlignment, MultiDomainDataset
from train import full_training, lmso_split, balanced_undersampling, subject_specific, multi_split_helper

import mne
import argparse
from mne.datasets import eegbci

import scipy.io as scio
import os
import numpy as np

/home/msai/anishmad001/.conda/envs/py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TOPLEVEL_EEGBCI = TOPLEVEL
BAD_SUBJECTS_EEGBCI = [87, 89, 91, 99]
SUBJECTS_EEGBCI = list(i for i in range(109) if i not in BAD_SUBJECTS_EEGBCI)
EVENTS_EEGBCI = dict(hands=2, feet=3)
BASELINE_EYES_OPEN = [1]
BASELINE_EYES_CLOSED = [2]

MOTOR_FISTS = (3, 7, 11)
IMAGERY_FISTS = (4, 8, 12)
MOTOR_FEET = (5, 9, 13)
IMAGERY_FEET_V_FISTS = (6, 10, 14)

In [2]:
targets=4
tmin=0
tlen=3
t_ev=0
t_sub=None
normalizer=zscore
low_f=None
high_f=None
alignment=False
all_events = []

result_path = '/home/msai/anishmad001/codes/EEG-Conformer/data/phisionet/'

In [5]:
paths = [eegbci.load_data(s+1, IMAGERY_FISTS, path=str(TOPLEVEL_EEGBCI), update_path=False) for s in SUBJECTS_EEGBCI]
raws = [mne.io.concatenate_raws([mne.io.read_raw_edf(p, preload=True) for p in path])
        for path in tqdm.tqdm(paths, unit='subj', desc='Loading')]
# datasets = OrderedDict()

for i, raw in tqdm.tqdm(list(zip(SUBJECTS_EEGBCI, raws)), desc='Preprocessing'):
    if raw.info['sfreq'] != 160:
        tqdm.tqdm.write('Skipping..., sampling frequency: {}'.format(raw.info['sfreq']))
        continue
    raw.rename_channels(lambda x: x.strip('.'))
    if low_f or high_f:
        raw.filter(low_f, high_f, fir_design='firwin', skip_by_annotation='edge')
    events, _ = mne.events_from_annotations(raw, event_id=dict(T1=0, T2=1))
    all_events.append(events)
    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
    epochs = mne.Epochs(raw, events[:41, ...], tmin=tmin, tmax=tmin + tlen - 1 / raw.info['sfreq'], picks=picks,
                        baseline=None, reject_by_annotation=False)#.drop_bad()
    if targets > 2:
        paths = eegbci.load_data(i + 1, BASELINE_EYES_OPEN, path=str(TOPLEVEL_EEGBCI), update_path=False)
        raw = mne.io.concatenate_raws([mne.io.read_raw_edf(p, preload=True) for p in paths])
        raw.rename_channels(lambda x: x.strip('.'))
        if low_f or high_f:
            raw.filter(low_f, high_f, fir_design='firwin', skip_by_annotation='edge')
        events = np.zeros((events.shape[0] // 2, 3)).astype('int')
        events[:, -1] = 2
        events[:, 0] = np.linspace(0, raw.info['sfreq'] * (60 - 2 * tlen), num=events.shape[0]).astype(np.int)
        all_events.append(events)
        picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
        eyes_epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmin + tlen - 1 / raw.info['sfreq'], picks=picks,
                                 baseline=None, reject_by_annotation=False)#.drop_bad()
        epochs = mne.concatenate_epochs([eyes_epochs, epochs])
    if targets > 3:
        paths = eegbci.load_data(i+1, IMAGERY_FEET_V_FISTS, path=str(TOPLEVEL_EEGBCI), update_path=False)
        raw = mne.io.concatenate_raws([mne.io.read_raw_edf(p, preload=True) for p in paths])
        raw.rename_channels(lambda x: x.strip('.'))
        if low_f or high_f:
            raw.filter(low_f, high_f, fir_design='firwin', skip_by_annotation='edge')
        events, _ = mne.events_from_annotations(raw, event_id=dict(T2=3))
        all_events.append(events)

        picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')
        feet_epochs = mne.Epochs(raw, events[:20, ...], tmin=tmin, tmax=tmin + tlen - 1 / raw.info['sfreq'],
                                 picks=picks, baseline=None, reject_by_annotation=False)#.drop_bad()
        epochs = mne.concatenate_epochs([epochs, feet_epochs])
    
    data = epochs.get_data()
    data = np.transpose(data, (2, 1, 0))
    epoch_events = epochs.events[:, 2]
    labels = np.reshape(epoch_events, (epoch_events.shape[0], 1))
    mat = {'data': data, 'label': labels}
    scio.savemat(os.path.join(result_path, str(i) + ".mat"), mat)

Loading:   0%|          | 0/105 [00:00<?, ?subj/s]

Extracting EDF parameters from /home/msai/anishmad001/codes/bci_data/rawfif_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/msai/anishmad001/codes/bci_data/rawfif_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/msai/anishmad001/codes/bci_data/rawfif_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


Loading:   1%|          | 1/105 [00:00<00:12,  8.40subj/s]

Extracting EDF parameters from /home/msai/anishmad001/codes/bci_data/rawfif_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S002/S002R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/msai/anishmad001/codes/bci_data/rawfif_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S002/S002R08.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/msai/anishmad001/codes/bci_data/rawfif_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S002/S002R12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...


Loading:   2%|▏         | 2/105 [00:00<00:11,  8.65subj/s]

...

/tmp/ipykernel_939073/3586428947.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  events[:, 0] = np.linspace(0, raw.info['sfreq'] * (60 - 2 * tlen), num=events.shape[0]).astype(np.int)
/tmp/ipykernel_939073/3586428947.py:31: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([eyes_epochs, epochs])


Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/msai/anishmad001/codes/bci_data/rawfif_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S109/S109R10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Extracting EDF parameters from /home/msai/anishmad001/codes/bci_data/rawfif_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S109/S109R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Used Annotations descriptions: ['T2']
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_939073/3586428947.py:44: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([epochs, feet_epochs])


Using data from preloaded Raw for 20 events and 480 original time points ...
Not setting metadata
83 matching events found
No baseline correction applied


Preprocessing: 100%|██████████| 105/105 [00:31<00:00,  3.31it/s]


In [8]:
labels

array([[2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3]])

In [9]:
np.unique(labels)

array([0, 1, 2, 3])

In [10]:
idx = np.where(labels == 0)

In [14]:
len(labels)

83

In [15]:
data.shape

(480, 64, 83)

In [11]:
sub = 5
total_data = scio.loadmat(os.path.join(result_path, '%d.mat' % sub))
train_data = total_data['data']
train_label = total_data['label']

In [12]:
train_data.shape

(480, 64, 83)